In [ ]:
import os
import openai
import glob

# Setup OpenAI API
openai.api_key = os.getenv("OPENAI_API_KEY")

# Define paths
transcripts_dir = 'data/transcripts'
qa_youtube_dir = 'data/qa_youtube'
if not os.path.exists(qa_youtube_dir):
    os.makedirs(qa_youtube_dir)

# Function to summarize and generate questions
def process_transcript(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        transcript = file.read()
    
    # Request GPT-4 for summary and questions
    response = openai.Completion.create(
        model="gpt-4",
        prompt=f"Please summarize the following transcript and generate a relevant question:\n\n{transcript}",
        max_tokens=500,
        temperature=0.7
    )
    
    # Extract summary and question
    summary = response.choices[0].text.strip().split('\n')[0]
    question = response.choices[0].text.strip().split('\n')[1] if len(response.choices[0].text.strip().split('\n')) > 1 else 'No question generated'
    
    return summary, question

# Process all transcript files
for transcript_file in glob.glob(os.path.join(transcripts_dir, '*.txt')):
    filename = os.path.basename(transcript_file)
    title, _ = os.path.splitext(filename)
    
    summary, question = process_transcript(transcript_file)
    
    # Save summary and question to new files
    with open(os.path.join(qa_youtube_dir, f'{title}_summary.txt'), 'w', encoding='utf-8') as summary_file:
        summary_file.write(summary)
    
    with open(os.path.join(qa_youtube_dir, f'{title}_question.txt'), 'w', encoding='utf-8') as question_file:
        question_file.write(question)
    
    # Optionally, save the original transcript as well
    with open(os.path.join(qa_youtube_dir, f'{title}_transcript.txt'), 'w', encoding='utf-8') as transcript_file_copy:
        with open(transcript_file, 'r', encoding='utf-8') as original_file:
            transcript_file_copy.write(original_file.read())

print("Processing complete. Files saved in 'data/qa_youtube'.")